In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters:", len(text))
print(text[:500])

length of dataset in characters: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [2]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
def encode(s): return [stoi[ch] for ch in s]
def decode(l): return ''.join([itos[i] for i in l])
print(encode('hii there'))
print(decode(encode('hii there')))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [3]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:50])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56])


In [4]:
segmentation_ratio = 0.9
n = int(segmentation_ratio * len(data))
train_data = data[:n]
val_data = data[n:]

In [5]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when context is {context.tolist()} target is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when context is [24] target is 43
when context is [24, 43] target is 58
when context is [24, 43, 58] target is 5
when context is [24, 43, 58, 5] target is 57
when context is [24, 43, 58, 5, 57] target is 1
when context is [24, 43, 58, 5, 57, 1] target is 46
when context is [24, 43, 58, 5, 57, 1, 46] target is 43
when context is [24, 43, 58, 5, 57, 1, 46, 43] target is 39
when context is [44] target is 53
when context is [44, 53] target is 56
when context is [44, 53, 56] target is 1
when context is [44, 53, 56, 1] target is 58
when context is [44, 53, 56, 1, 58] target is 46
when context is [44

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, indices, targets=None):
        # Get some indices, embed them into a tensor
        # with the shape (indices.shape, vocab_size), and return the result
        logits = self.token_embedding_table(indices)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # Get an index with the shape (batch, dimension)
        for _ in range(max_new_tokens):
            # Calculate the logits with a shape of (batch, dimension, vocab_size), retain only the last dimension (because of 'BIGRAM'!), and use it to determine the next index
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # becomes (batch, vocab_size)
            probs = F.softmax(logits, dim=1)  # (batch, vocab_size)
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (batch, dimension+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros(
    (3, 2), dtype=torch.long), max_new_tokens=10)[0].tolist()))
print(decode(m.generate(idx=torch.zeros(
    (3, 2), dtype=torch.long), max_new_tokens=10)[1].tolist()))
print(decode(m.generate(idx=torch.zeros(
    (3, 2), dtype=torch.long), max_new_tokens=10)[2].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


Sr?qP-QWkt


pfqoioCp$N


R:aeOvERms


In [7]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [8]:
batch_size = 32

for steps in range(10000):

    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4607205390930176


In [9]:
# 2.4607205390930176

In [10]:
print(decode(m.generate(idx=torch.zeros(
    (1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


TIUEd ce w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
a


In [11]:
#
# TIUEd ce w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
# Rie hicomyonthar's
# Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
# KIN d pe wither vouprrouthercc.
# hathe; d!
# My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
# h hay.JUCle n prids, r loncave w hollular s O:
# HIs; ht anjx?

# DUThinqunt.

# LaZAnde.
# a